### face_detect_save_v1

In [2]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('audience.mp4')

# Initialize the Mediapipe face detection module
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection:
    num_detection = 0  # Initialize the counter for the number of detections
    prev_faces = []  # List to store the previously detected face coordinates

    while cap.isOpened() and num_detection < 30:  # Break the loop after 30 detections
        success, frame = cap.read()
        if not success:
            break

        target_frame = frame.copy()  # copy frame

#         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         ret, otsu = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#         contours, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

#         COLOR = (0, 200, 0)  # green

        # Perform face detection using Mediapipe on the current frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw face detection annotations on the frame and crop card region
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)

                # Get the bounding box coordinates of the detected face
                bbox = detection.location_data.relative_bounding_box
                xmin, ymin, width, height = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), \
                                           int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])

                # Check if the current face overlaps with any previously detected faces
                is_duplicate = False
                for prev_face in prev_faces:
                    if abs(xmin - prev_face[0]) < prev_face[2] and abs(ymin - prev_face[1]) < prev_face[3]:
                        is_duplicate = True
                        break

                if not is_duplicate:
                    num_detection += 1

                    # Crop the face region based on the face bounding box
                    face_crop = frame[ymin:ymin + height, xmin:xmin + width]
                    cv2.imshow(f'face_crop_{num_detection}', face_crop)
                    cv2.imwrite(f'face_crop_{num_detection}.png', face_crop)

                    # Add the current face coordinates to the list of previously detected faces
                    prev_faces.append((xmin, ymin, width, height))

        cv2.imshow('frame', frame)
#         cv2.imshow('contour', target_frame)

        if cv2.waitKey(1) == ord('q'):  # exit when 'q' is pressed
            break

cap.release()
cv2.destroyAllWindows()

print("Number of people detected:", num_detection)


Number of people detected: 12


### Apply ProcessPoolExecutor

In [5]:
import cv2
import os
import threading
import mediapipe as mp
from concurrent.futures import ProcessPoolExecutor

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('audience.mp4')

def cpu_bound_func(cap):
    print(f"{os.getpid()} process | {threading.get_ident()} thread, {cap}")
    numbers = range(1, cap)
    total = 1

def main():
    executor = ProcessPoolExecutor(max_workers=10)
    results = list(executor.map(cpu_bound_func, cap))
    print(results)

# Initialize the Mediapipe face detection module
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection:
    num_detection = 0  # Initialize the counter for the number of detections
    prev_faces = []  # List to store the previously detected face coordinates

    while cap.isOpened() and num_detection < 30:  # Break the loop after 30 detections
        success, frame = cap.read()
        if not success:
            break

        target_frame = frame.copy()  # copy frame

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, otsu = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        contours, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

        COLOR = (0, 200, 0)  # green

        # Perform face detection using Mediapipe on the current frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw face detection annotations on the frame and crop card region
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)

                # Get the bounding box coordinates of the detected face
                bbox = detection.location_data.relative_bounding_box
                xmin, ymin, width, height = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), \
                                           int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])

                # Check if the current face overlaps with any previously detected faces
                is_duplicate = False
                for prev_face in prev_faces:
                    if abs(xmin - prev_face[0]) < prev_face[2] and abs(ymin - prev_face[1]) < prev_face[3]:
                        is_duplicate = True
                        break

                if not is_duplicate:
                    num_detection += 1

                    # Crop the face region based on the face bounding box
                    face_crop = frame[ymin:ymin + height, xmin:xmin + width]
                    cv2.imshow(f'face_crop_{num_detection}', face_crop)
                    cv2.imwrite(f'face_crop_{num_detection}.png', face_crop)

                    # Add the current face coordinates to the list of previously detected faces
                    prev_faces.append((xmin, ymin, width, height))

        cv2.imshow('frame', frame)
        # cv2.imshow('frame', cv2.resize(image, None, fx=0.5, fy=0.5))
    
        if cv2.waitKey(1) == ord('q'):  # exit when 'q' is pressed
            break

cap.release()
cv2.destroyAllWindows()

print("Number of detected people:", num_detection)


Number of detected people: 12


### Apply ThreadPoolExecutor

In [3]:
import cv2
import os
import threading
import mediapipe as mp
from concurrent.futures import ThreadPoolExecutor

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture('audience.mp4')

def cpu_bound_func(cap):
    print(f"{os.getpid()} process | {threading.get_ident()} thread, {cap}")
    numbers = range(1, cap)
    total = 1

def main():
    executor = ThreadPoolExecutor(max_workers=10)
    results = list(executor.map(cpu_bound_func, cap))
    print(results)

# Initialize the Mediapipe face detection module
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7) as face_detection:
    num_detection = 0  # Initialize the counter for the number of detections
    prev_faces = []  # List to store the previously detected face coordinates

    while cap.isOpened() and num_detection < 30:  # Break the loop after 30 detections
        success, frame = cap.read()
        if not success:
            break

        target_frame = frame.copy()  # copy frame

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, otsu = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        contours, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

        COLOR = (0, 200, 0)  # green

        # Perform face detection using Mediapipe on the current frame
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw face detection annotations on the frame and crop card region
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)

                # Get the bounding box coordinates of the detected face
                bbox = detection.location_data.relative_bounding_box
                xmin, ymin, width, height = int(bbox.xmin * frame.shape[1]), int(bbox.ymin * frame.shape[0]), \
                                           int(bbox.width * frame.shape[1]), int(bbox.height * frame.shape[0])

                # Check if the current face overlaps with any previously detected faces
                is_duplicate = False
                for prev_face in prev_faces:
                    if abs(xmin - prev_face[0]) < prev_face[2] and abs(ymin - prev_face[1]) < prev_face[3]:
                        is_duplicate = True
                        break

                if not is_duplicate:
                    num_detection += 1

                    # Crop the face region based on the face bounding box
                    face_crop = frame[ymin:ymin + height, xmin:xmin + width]
                    cv2.imshow(f'face_crop_{num_detection}', face_crop)
                    cv2.imwrite(f'face_crop_{num_detection}.png', face_crop)

                    # Add the current face coordinates to the list of previously detected faces
                    prev_faces.append((xmin, ymin, width, height))

        cv2.imshow('frame', frame)
#         cv2.imshow('contour', target_frame)

        if cv2.waitKey(1) == ord('q'):  # exit when 'q' is pressed
            break

cap.release()
cv2.destroyAllWindows()

print("Number of people detected:", num_detection)


Number of people detected: 12


비디오 파일 프레임 단위 분석 -> 프레임 단위로 이미지 추출 -> 이미지 픽셀 단위 분석 -> 